## Imports

In [1]:
from params import FuzzyInputVariable_List_Trapezoids
import numpy as np
import matplotlib.pyplot as plt
from ANFIS import ANFIS
import time
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from optimizers.swarm import SwarmOptimizer, Phase
from optimizers.default import DefaultOptimizer
from optimizers.genetic import GeneticOptimizer, SmallestMaeErrorFitness, MultiPointCrossing, NRandomChangesMutation, RouletteWheelSelection, RankSelection

## Reloading imports

In [2]:
%load_ext autoreload

In [3]:
%autoreload

## Generating data

In [7]:
def f(x, y, z):
        return x*y + x*z + y*z

In [8]:
def generate_data(domain, step=0.1):
    var = np.arange(domain[0], domain[1], step)
    x, y, z = np.meshgrid(var, var, var)

    x = x.flatten()
    y = y.flatten()
    z = z.flatten()

    labels = np.round(x) * np.round(y) * np.round(z)
    data = np.column_stack((x, y, z))

    return data, labels

In [12]:
data_xyz, labels = generate_data(domain=(0, 1), step=0.5)

## Creating ANFIS

In [19]:
# create anfis
mf1 = [[0.25+i*0.5, 0.5, 0.5, 0.5] for i in range(2)]
var_names = [str(i) for i in range(2)]

varX = FuzzyInputVariable_List_Trapezoids(mf1, "XAxis", var_names)
varY = FuzzyInputVariable_List_Trapezoids(mf1, "YAxis", var_names)
varZ = FuzzyInputVariable_List_Trapezoids(mf1, "YAxis", var_names)

anfis = ANFIS([varX, varY, varZ], data_xyz.T, labels)

## Training

In [14]:
optimizer = DefaultOptimizer(learn_operators=False)

start = time.time()
anfis.train(optimizer)
elapsed_time = time.time() - start

print(f'Training time: {elapsed_time:.2f}')

Optimization finished after 2.49s
Training time: 2.49


## Evaluation

In [20]:
# evaluate
y_pred = anfis.estimate_labels().flatten()
error = np.sum(np.abs(y_pred - labels))
print(f'MAE Error: {error:.2f}')

MAE Error: 7.89
